In [1]:
import sys
sys.path.append('/host/d/Github')
import os
import numpy as np
import pandas as pd
import nibabel as nb
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
import Osteosarcoma.functions_collection as ff
import Osteosarcoma.Build_lists.Build_list as Build_list

from __future__ import annotations

import os  # needed navigate the system to get the input data

import radiomics
from radiomics import (
    featureextractor,  # This module is used for interaction with pyradiomics
)

### patchify image

In [4]:
build = Build_list.Build(os.path.join('/host/d/Data/Habitats/Jishuitan/Patient_lists', 'labels_with_image_info_included.xlsx'))
batch_list, patient_index_list, label_list, image_path_list, mask_path_list = build.__build__()
print(f'Number of cases to process: {len(image_path_list)}')

resampled_path = '/host/d/Data/Habitats/Jishuitan/resampled_data/'

Number of cases to process: 81


In [ ]:
for index in range(len(patient_index_list)):
    patient_index = patient_index_list[index]
    image_path = os.path.join(resampled_path, str(patient_index), 'image_resampled.nii.gz')
    label_path = os.path.join(resampled_path, str(patient_index), 'label_resampled.nii.gz')
    print('image path:', image_path, '\nlabel path:', label_path)

    habitats_out_path = '/host/d/Data/Habitats/Jishuitan/habitats/'
    ff.make_folder([os.path.join(habitats_out_path, str(patient_index))])
    
    label_nii = nb.load(label_path)
    label_arr = label_nii.get_fdata()
    affine = label_nii.affine; header = label_nii.header

    # bbox
    bbox_out_path = os.path.join(habitats_out_path, str(patient_index), 'tumor_bbox.nii.gz')
    bbox_arr, x0, x1, y0, y1, z0, z1 = bbox3d(label_arr, buffer_x=5, buffer_y = 5, buffer_z = 1)
    bbox_nii = nb.Nifti1Image(bbox_arr, affine=affine, header=header)
    nb.save(bbox_nii, bbox_out_path)

    # extract patches
   
    patch_size_mm = 10.0               # 1 cm
    min_tumor_fraction = 0.50          # 至少 30% tumor 才保留
    out_dir = os.path.join(habitats_out_path, str(patient_index), "patches")
    os.makedirs(out_dir, exist_ok=True)

    patch_table_path = os.path.join(habitats_out_path, str(patient_index), "patch_table.xlsx")

    # 你上一步已经得到 bbox_arr（shape = (X,Y,Z)），以及 bbox 的坐标 x0,x1,y0,y1,z0,z1
    # 这里假设你已经有：bbox_arr, x0,x1,y0,y1,z0,z1
    # 并且 label_path 指向 label_resampled.nii.gz

    patchify(label_arr, x0, x1, y0, y1, z0, z1, patch_size_mm, min_tumor_fraction, out_dir, patch_table_path)


    
   


image path: /host/d/Data/Habitats/Jishuitan/resampled_data/5/image_resampled.nii.gz 
label path: /host/d/Data/Habitats/Jishuitan/resampled_data/5/label_resampled.nii.gz
Tight bbox (x,y,z): (188, 353) (165, 331) (39, 101)
Buffered bbox: x[183:358], y[160:336], z[38:102]
Spacing (mm): 0.390625 0.3906250298023224 1.0
Patch size (vox): 26 26 10
Grid start: (183, 160, 38)
Grid end  : (364, 341, 107)
Grid n patches: (7, 7, 7)
image path: /host/d/Data/Habitats/Jishuitan/resampled_data/7/image_resampled.nii.gz 
label path: /host/d/Data/Habitats/Jishuitan/resampled_data/7/label_resampled.nii.gz
Tight bbox (x,y,z): (115, 332) (160, 377) (18, 115)
Buffered bbox: x[110:337], y[155:382], z[17:116]
Spacing (mm): 0.34734854102134705 0.34734851121902466 1.0
Patch size (vox): 29 29 10
Grid start: (110, 155, 17)
Grid end  : (341, 386, 116)
Grid n patches: (8, 8, 10)


In [16]:
def bbox3d(label, buffer_x=5, buffer_y=5, buffer_z=5):
    tumor = (label_arr == 1)

    if not np.any(tumor):
        raise ValueError(f"No tumor voxels found in label: {label_path}")

    # ---------- 2) find tight bbox ----------
    xx, yy, zz = np.where(tumor)

    x_min, x_max = xx.min(), xx.max()
    y_min, y_max = yy.min(), yy.max()
    z_min, z_max = zz.min(), zz.max()

    # ---------- 3) apply buffer and clip ----------
    X, Y, Z = label_arr.shape

    x0 = max(x_min - buffer_x, 0)
    x1 = min(x_max + buffer_x, X - 1)

    y0 = max(y_min - buffer_y, 0)
    y1 = min(y_max + buffer_y, Y - 1)

    z0 = max(z_min - buffer_z, 0)
    z1 = min(z_max + buffer_z, Z - 1)

    print("Tight bbox (x,y,z):",
        (x_min, x_max), (y_min, y_max), (z_min, z_max))
    print("Buffered bbox:",
        f"x[{x0}:{x1}], y[{y0}:{y1}], z[{z0}:{z1}]")

    # ---------- 4) build bbox mask ----------
    bbox_arr = np.zeros_like(label_arr, dtype=np.uint8)
    bbox_arr[x0:x1+1, y0:y1+1, z0:z1+1] = 1

    return bbox_arr, x0, x1, y0, y1, z0, z1

In [13]:
### bounding box extraction
# ---------- inputs ----------
buffer_x = 5   # voxels
buffer_y = 5
buffer_z = 3

habitats_out_path = '/host/d/Data/Habitats/Jishuitan/habitats/'
bbox_out_path = os.path.join(habitats_out_path, str(patient_index), 'tumor_bbox.nii.gz')
ff.make_folder([os.path.dirname(bbox_out_path)])

# ---------- 1) load label ----------
label_nii = nb.load(label_path)
label_arr = label_nii.get_fdata()     # shape: (X, Y, Z) in nibabel convention

# 注意：nibabel array 顺序是 (x, y, z)
tumor = (label_arr == 1)

if not np.any(tumor):
    raise ValueError(f"No tumor voxels found in label: {label_path}")

# ---------- 2) find tight bbox ----------
xx, yy, zz = np.where(tumor)

x_min, x_max = xx.min(), xx.max()
y_min, y_max = yy.min(), yy.max()
z_min, z_max = zz.min(), zz.max()

# ---------- 3) apply buffer and clip ----------
X, Y, Z = label_arr.shape

x0 = max(x_min - buffer_x, 0)
x1 = min(x_max + buffer_x, X - 1)

y0 = max(y_min - buffer_y, 0)
y1 = min(y_max + buffer_y, Y - 1)

z0 = max(z_min - buffer_z, 0)
z1 = min(z_max + buffer_z, Z - 1)

print("Tight bbox (x,y,z):",
      (x_min, x_max), (y_min, y_max), (z_min, z_max))
print("Buffered bbox:",
      f"x[{x0}:{x1}], y[{y0}:{y1}], z[{z0}:{z1}]")

# ---------- 4) build bbox mask ----------
bbox_arr = np.zeros_like(label_arr, dtype=np.uint8)
bbox_arr[x0:x1+1, y0:y1+1, z0:z1+1] = 1

# ---------- 5) save as nii.gz ----------
bbox_nii = nb.Nifti1Image(bbox_arr, affine=label_nii.affine, header=label_nii.header)
nb.save(bbox_nii, bbox_out_path)


Tight bbox (x,y,z): (188, 353) (165, 331) (39, 101)
Buffered bbox: x[183:358], y[160:336], z[36:104]


In [18]:
### patchify


# 你上一步已经得到 bbox_arr（shape = (X,Y,Z)），以及 bbox 的坐标 x0,x1,y0,y1,z0,z1
# 这里假设你已经有：bbox_arr, x0,x1,y0,y1,z0,z1
# 并且 label_path 指向 label_resampled.nii.gz




def patchify(label_arr, x0, x1, y0, y1, z0, z1, patch_size_mm, min_tumor_fraction, out_dir, patch_table_path):
    tumor = (label_arr == 1)

    # spacing from header (in mm)
    # nibabel: header['pixdim'] = [?, dx, dy, dz, ...]
    pixdim = label_nii.header.get_zooms()[:3]  # (dx,dy,dz)
    dx, dy, dz = float(pixdim[0]), float(pixdim[1]), float(pixdim[2])

    print("Spacing (mm):", dx, dy, dz)

    # ---------------- compute patch size in voxels ----------------
    nx = int(np.ceil(patch_size_mm / dx))
    ny = int(np.ceil(patch_size_mm / dy))
    nz = int(np.ceil(patch_size_mm / dz))

    # 强制至少 1 voxel
    nx = max(nx, 1)
    ny = max(ny, 1)
    nz = max(nz, 1)

    print("Patch size (vox):", nx, ny, nz)

    X, Y, Z = label_arr.shape

    # ---------------- determine grid ranges (allow extend beyond bbox if not divisible) ----------------
    bbox_len_x = (x1 - x0 + 1)
    bbox_len_y = (y1 - y0 + 1)
    bbox_len_z = (z1 - z0 + 1)

    npx = int(np.ceil(bbox_len_x / nx))
    npy = int(np.ceil(bbox_len_y / ny))
    npz = int(np.ceil(bbox_len_z / nz))

    # grid_end may extend outside bbox; later clip to image boundary
    grid_x_end = x0 + npx * nx - 1
    grid_y_end = y0 + npy * ny - 1
    grid_z_end = z0 + npz * nz - 1

    # clip to image bounds
    grid_x_end = min(grid_x_end, X - 1)
    grid_y_end = min(grid_y_end, Y - 1)
    grid_z_end = min(grid_z_end, Z - 1)

    print("Grid start:", (x0, y0, z0))
    print("Grid end  :", (grid_x_end, grid_y_end, grid_z_end))
    print("Grid n patches:", (npx, npy, npz))

    # ---------------- iterate patches ----------------
    rows = []
    patch_id = 0

    for iz in range(npz):
        for iy in range(npy):
            for ix in range(npx):
                # patch bounds in voxel index (inclusive)
                px0 = x0 + ix * nx
                py0 = y0 + iy * ny
                pz0 = z0 + iz * nz

                px1 = px0 + nx - 1
                py1 = py0 + ny - 1
                pz1 = pz0 + nz - 1

                # clip to image bounds (important when extended)
                px0_c = max(px0, 0); py0_c = max(py0, 0); pz0_c = max(pz0, 0)
                px1_c = min(px1, X-1); py1_c = min(py1, Y-1); pz1_c = min(pz1, Z-1)

                # if patch completely outside image (rare), skip
                if (px0_c > px1_c) or (py0_c > py1_c) or (pz0_c > pz1_c):
                    continue

                # tumor fraction inside THIS patch (using tumor voxels only)
                patch_tumor = tumor[px0_c:px1_c+1, py0_c:py1_c+1, pz0_c:pz1_c+1]
                patch_voxels = patch_tumor.size
                tumor_voxels = int(patch_tumor.sum())
                tumor_fraction = tumor_voxels / float(patch_voxels)

                if tumor_fraction < min_tumor_fraction:
                    continue

                # ---------------- build and save patch mask: (patch region) AND (tumor) ----------------
                patch_mask = np.zeros_like(tumor, dtype=np.uint8)
                patch_mask[px0_c:px1_c+1, py0_c:py1_c+1, pz0_c:pz1_c+1] = patch_tumor.astype(np.uint8)

                patch_mask_nii = nb.Nifti1Image(patch_mask, affine=label_nii.affine, header=label_nii.header)
                patch_mask_path = os.path.join(out_dir, f"patch_{patch_id:04d}.nii.gz")
                nb.save(patch_mask_nii, patch_mask_path)

                # ---------------- record 8 vertices (voxel coords) ----------------
                # 顶点定义（按 voxel index，inclusive）：
                # z0 面： (x0,y0,z0) (x1,y0,z0) (x1,y1,z0) (x0,y1,z0)
                # z1 面： (x0,y0,z1) (x1,y0,z1) (x1,y1,z1) (x0,y1,z1)
                v = [
                    (px0_c, py0_c, pz0_c),
                    (px1_c, py0_c, pz0_c),
                    (px1_c, py1_c, pz0_c),
                    (px0_c, py1_c, pz0_c),
                    (px0_c, py0_c, pz1_c),
                    (px1_c, py0_c, pz1_c),
                    (px1_c, py1_c, pz1_c),
                    (px0_c, py1_c, pz1_c),
                ]

                row = {
                    "patch_id": patch_id,
                    "tumor_fraction": tumor_fraction,
                    "mask_path": patch_mask_path,  # 可选：方便后续 radiomics batch
                }
                for i, (vx, vy, vz) in enumerate(v, start=1):
                    row[f"x_{i}"] = int(vx)
                    row[f"y_{i}"] = int(vy)
                    row[f"z_{i}"] = int(vz)

                rows.append(row)
                patch_id += 1

    # print("Kept patches:", patch_id)

    # ---------------- save table ----------------
    df_patches = pd.DataFrame(rows)
    df_patches.to_excel(patch_table_path, index=False)
    # print("Saved patch table:", patch_table_path)


### initiate extractor

In [2]:
# Instantiate the extractor
paramPath = '/host/d/Github/Osteosarcoma/radiomics_settings/MR_setting_1.yaml'
extractor = featureextractor.RadiomicsFeatureExtractor(paramPath)

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)
print('Enabled features:\n\t', extractor.enabledFeatures)

Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': True, 'normalizeScale': 100, 'removeOutliers': None, 'resampledPixelSpacing': [1, 1, 1], 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 10, 'voxelArrayShift': 300, 'geometryTolerance': 0.0001}
Enabled filters:
	 {'Original': {}, 'LoG': {'sigma': [2.0, 4.0, 6.0]}, 'Wavelet': {}}
Enabled features:
	 {'shape': None, 'firstorder': None, 'glcm': ['Autocorrelation', 'JointAverage', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'JointEnergy', 'JointEntropy', 'Imc1', 'Imc2', 'Idm', 'Idmn', 'Id', 'Idn', 'InverseVariance', 'MaximumProbability', 'SumEntropy', 'SumSquares'], 'glrlm': None, 'glszm': None, 'gldm': None, 'ngtdm': None}


### define patient list

In [9]:
build = Build_list.Build(os.path.join('/host/d/Data/Habitats/Jishuitan/Patient_lists', 'labels_with_image_info_seg_reader2.xlsx'))
batch_list, patient_index_list, label_list, image_path_list, mask_path_list = build.__build__()
print(f'Number of cases to process: {len(image_path_list)}')

Number of cases to process: 30


### extract features

In [10]:
rows = []
for i in range(0,len(patient_index_list)):
    img_p = image_path_list[i]
    msk_p = mask_path_list[i]
    cid = patient_index_list[i]
    print('i', i, ' image path:', img_p, 'mask path:', msk_p)

    result = extractor.execute(img_p, msk_p)

    # Keep only radiomics features (drop diagnostics)
    feats = {k: v for k, v in result.items() if not k.startswith("diagnostics_")}
    feats["Patient_index"] = cid
    feats["Image_filepath"] = img_p
    feats["Mask_filepath"] = msk_p

    rows.append(feats)

i 0  image path: /host/d/Data/Habitats/Jishuitan/original_data/1/img.nii.gz mask path: /host/d/Data/Habitats/Jishuitan/original_data/1/label_reader2.nii.gz
i 1  image path: /host/d/Data/Habitats/Jishuitan/original_data/5/img.nii.gz mask path: /host/d/Data/Habitats/Jishuitan/original_data/5/label_reader2.nii.gz
i 2  image path: /host/d/Data/Habitats/Jishuitan/original_data/7/img.nii.gz mask path: /host/d/Data/Habitats/Jishuitan/original_data/7/label_reader2.nii.gz
i 3  image path: /host/d/Data/Habitats/Jishuitan/original_data/8/img.nii.gz mask path: /host/d/Data/Habitats/Jishuitan/original_data/8/label_reader2.nii.gz
i 4  image path: /host/d/Data/Habitats/Jishuitan/original_data/11/img.nii.gz mask path: /host/d/Data/Habitats/Jishuitan/original_data/11/label_reader2.nii.gz
i 5  image path: /host/d/Data/Habitats/Jishuitan/original_data/15/img.nii.gz mask path: /host/d/Data/Habitats/Jishuitan/original_data/15/label_reader2.nii.gz
i 6  image path: /host/d/Data/Habitats/Jishuitan/original_da

In [13]:
df = pd.DataFrame(rows)

# Put id/path columns first
front_cols = [c for c in ["Patient_index", "Image_filepath", "Mask_filepath"] if c in df.columns]
other_cols = [c for c in df.columns if c not in front_cols]
df = df[front_cols + other_cols]


# df.to_excel('/host/d/projects/Habitats/radiomics/radiomics_measurements.xlsx', index=False)

In [ ]:
non_feature_cols = ["Patient_index", "Image_filepath", "Mask_filepath"]
feature_names = [c for c in df.columns if c not in non_feature_cols]

feature_index = list(range(1, len(feature_names) + 1))

feature_table = pd.DataFrame({
    "feature_index": feature_index,
    "feature_name": feature_names
})

# feature_table.to_excel('/host/d/projects/Habitats/radiomics/radiomics_features_list.xlsx', index=False)

### normalize features

#### normalize for reader 1

In [ ]:
### normalize features
# df = pd.read_excel('/host/d/projects/Habitats/radiomics/radiomics_measurements.xlsx')
### normalize features to [0,1]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
non_feature_cols = ["Patient_index", "Image_filepath", "Mask_filepath"]
feature_cols = [c for c in df.columns if c not in non_feature_cols]
df_features = df[feature_cols]
df_features_scaled = pd.DataFrame(scaler.fit_transform(df_features), columns=feature_cols)
df_scaled = pd.concat([df[non_feature_cols], df_features_scaled], axis=1)
df_scaled.to_excel('/host/d/projects/Habitats/radiomics/radiomics_measurements_normalized.xlsx', index=False)

In [ ]:
feature_min = scaler.data_min_
feature_max = scaler.data_max_

feature_table = pd.read_excel('/host/d/projects/Habitats/results/radiomics_features_list.xlsx')
feature_table['feature_min'] = feature_min
feature_table['feature_max'] = feature_max
feature_table.to_excel('/host/d/projects/Habitats/results/radiomics_features_list.xlsx', index=False)

#### normalize for reader 2, will use data_min and data_max from reader 1 normalization

In [15]:
# for each feature, get the scaler.data_min_ and data_max_ from reader 1 normalization and use them to normalize reader 2 features
scale_df = pd.read_excel('/host/d/projects/Habitats/radiomics/radiomics_features_list.xlsx')
df_reader2 = df.copy()
feature_names = [c for c in df_reader2.columns if c not in non_feature_cols]
for feature in feature_names:
    f_min = scale_df.loc[scale_df['feature_name'] == feature, 'feature_min'].values[0]
    f_max = scale_df.loc[scale_df['feature_name'] == feature, 'feature_max'].values[0]
    df_reader2[feature] = (df_reader2[feature] - f_min) / (f_max - f_min)
df_reader2.to_excel('/host/d/projects/Habitats/radiomics/radiomics_measurements_normalized_reader2.xlsx', index=False)

